In [ ]:
import numpy as np
import pandas as pd

# Load Data

In [ ]:
from enfobench.datasets import ElectricityDemandDataset

ds = ElectricityDemandDataset("../data/electricity-demand")

In [ ]:
unique_ids = ds.metadata_subset.list_unique_ids()

In [ ]:
len(unique_ids)

In [ ]:
unique_id = unique_ids[0]
unique_id

In [ ]:
target, past_covariates, metadata = ds.get_data_by_unique_id(unique_id)

In [ ]:
target.info()

In [ ]:
past_covariates.info()

# Make external forecasts based on covariates

In [ ]:
from enfobench.datasets.utils import create_perfect_forecasts_from_covariates

perfect_forecasts = create_perfect_forecasts_from_covariates(
    past_covariates,
    horizon=pd.Timedelta("7 days"),
    step=pd.Timedelta("24 hour"),
)

In [ ]:
perfect_forecasts.info()

In [ ]:
perfect_forecasts.head()

In [ ]:
perfect_forecasts.tail()

# Create a Multivariate Dataset

In [ ]:
from enfobench import Dataset

multivariate_dataset = Dataset(
    target=target,
    past_covariates=past_covariates,
    metadata=metadata,
)

### Note:
- these datasets are cleaned and contain no missing values
- the target has a resolution of 30 minutes but the weather data has observations only every hour

# Create Model

In [ ]:
from enfobench import AuthorInfo, ModelInfo, ForecasterType
from enfobench.evaluation.utils import create_forecast_index


class ExampleModel:
    def __init__(self, param1: int):
        self.param1 = param1

    def info(self):
        return ModelInfo(
            name="TestModel",
            authors=[
                AuthorInfo("Attila Balint", email="optional"),
            ],
            type=ForecasterType.point,
            params={
                "param1": 1,
            },
        )

    def forecast(
        self,
        horizon: int,
        history: pd.DataFrame,
        past_covariates=None,
        future_covariates=None,
        metadata: dict | None = None,
        level=None,
        **kwargs,
    ):
        index = create_forecast_index(history, horizon)
        prediction = pd.DataFrame(
            index=index,
            data={
                "yhat": np.full(horizon, fill_value=history.y.mean()) + self.param1,
            },
        )
        return prediction

In [ ]:
model = ExampleModel(1)

# Perform crossvalidation on your model locally

In [ ]:
from enfobench.evaluation import cross_validate

crossval_df = cross_validate(
    model,
    multivariate_dataset,
    start_date=pd.Timestamp("2022-01-01T10:00:00"),
    end_date=pd.Timestamp("2022-02-01T00:00:00"),
    horizon=pd.Timedelta("38 hours"),
    step=pd.Timedelta("1 day"),
)

In [ ]:
crossval_df.head()

In [ ]:
cutoff_date_to_plot = crossval_df.cutoff_date.unique()[0]
crossval_df.loc[crossval_df.cutoff_date == cutoff_date_to_plot].set_index("timestamp").drop(
    columns=["cutoff_date"]
).plot()

# Evaluate metrics

In [ ]:
from enfobench.evaluation import evaluate_metrics
from enfobench.evaluation.metrics import mean_absolute_error, mean_bias_error

In [ ]:
evaluate_metrics(crossval_df, metrics={"MAE": mean_absolute_error, "MBE": mean_bias_error})

In [ ]:
metrics = evaluate_metrics(
    crossval_df,
    metrics={"MAE": mean_absolute_error, "MBE": mean_bias_error},
    groupby="cutoff_date",
)
metrics

In [ ]:
metrics[["MAE", "MBE"]].plot()

# Create external perfect forecast dataset to use as future covariates

In [ ]:
from enfobench.datasets.utils import create_perfect_forecasts_from_covariates

perfect_forecasts = create_perfect_forecasts_from_covariates(
    past_covariates,
    horizon=pd.Timedelta("7 days"),
    step=pd.Timedelta("12 hour"),
)

In [ ]:
perfect_forecasts.info()

In [ ]:
perfect_forecasts.head()

In [ ]:
perfect_forecasts.tail()

In [ ]:
full_dataset = Dataset(
    target=target,
    past_covariates=past_covariates,
    future_covariates=perfect_forecasts,
)

In [ ]:
crossval_df = cross_validate(
    ExampleModel(200),
    full_dataset,
    start_date=pd.Timestamp("2022-01-01T10:00:00"),
    end_date=pd.Timestamp("2022-02-01T00:00:00"),
    horizon=pd.Timedelta("38 hours"),
    step=pd.Timedelta("1 day"),
)

In [ ]:
cutoff_date_to_plot = crossval_df.cutoff_date.unique()[0]
crossval_df.loc[crossval_df.cutoff_date == cutoff_date_to_plot].set_index("timestamp").drop(
    columns=["cutoff_date"]
).plot()

Evaluate the metrics you are interested in.

In [ ]:
evaluate_metrics(crossval_df, metrics={"MAE": mean_absolute_error, "MBE": mean_bias_error})

In [ ]:
metrics = evaluate_metrics(
    crossval_df,
    metrics={"MAE": mean_absolute_error, "MBE": mean_bias_error},
    groupby="cutoff_date",
)
metrics

In [ ]:
metrics[["MAE", "MBE"]].plot()